In [40]:
import numpy as np
from matplotlib.pyplot import *
from scipy.sparse import diags  # Pour les matrices diagonales

# REMARQUE : éviter d'appeler une variable ou une fonction "lambda" car déjà prédéfini par python

# CE QUI SUIT EST UNE BASE POUR IMPLEMENTER LES FONCTIONS DES DEUX METHODES

# La fonction suivante permet de discrétiser un intervalle [0,1] en N+1 noeuds équidistants
# (c'est-à-dire on "tranche" notre colonne de terre en N+1 morceaux)

def intervalle_discret(N): # Pour commencer on peut prendre par exemple N=10
    x=np.linspace(0,1,N+1)
    return x



# La fonction suivante permet de calculer la constante alpha de l'équation différentielle 
# On détermine alpha grâce aux valeurs de la conductivité thermique du sol et à sa capacité thermique volumique
# ATTENTION: il faut bien utiliser les unités SI

def calcul_alpha(lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol):
    alpha=lambda_conductivite_thermique_sol/capacite_thermique_volumique_sol
    return alpha

# La fonction suivante permet de calculer la valeur de lambda qui servira dans le calcul des températures de la colonne au cours du temps

def calcul_lambda(alpha,delta_x,delta_t):
    lamb=alpha*delta_t/((delta_x)**2)
    return lamb

# La fonction suivante permet d'initialiser les températures de notre colonne de terre à l'instant t=0 dans un vecteur colonne de dimensions N-1 (et pas N attention)
# Il s'agit d'un choix arbitraire étant donné que les valeurs vont toujours finir par converger vers les températures à l'équilibre thermodynamique de la colonne de terre 
# On choisit ici d'initialiser notre colonne avec un température uniforme qui vaut 0

def initialisation_T(N,T0):
    T=T0*np.ones((N-1,1)) # création d'un vecteur colonne avec que des zéros
    return T
    
# La fonction suivante permet de créer le vecteur qui sera utilisé pour calculer le vecteur T à l'étape k+1
# Il n'y a pas vraiment de nom pertinent pour cette fonction
# extremite fait référence aux températures aux extrémités de la colonne de terre (car elles interviennent uniquement dans ce vecteur)

def extremite(N,lamb,Tg,Td):
    ext=np.zeros((N-1,1))
    ext[0]=lamb*Tg
    ext[-1]=lamb*Td
    
    return ext


# On passe maintenant aux fonctions spécifiques à chaque méthode
# "e" correspond à explicite et "i" à implicite 

# METHODE EXPLICITE

# La fonction suivante permet de créer la matrice diagonale de taille (N-1)*(N-1)
# Cette matrice sera utilisée pour calculer le vecteur T à l'étape k+1 avec la méthode explicite
# ATENTION : la matrice diagonale est différente pour la méthode implicite

def matrice_diag_e(N,lamb):
    k = np.array([lamb*np.ones(N-2),1-2*lamb*np.ones(N-1),lamb*np.ones(N-2)])
    offset = [-1,0,1]
    A = diags(k,offset).toarray()
    return A

# La fonction suivante est l'implémentation de la méthode explicite
# Cette fonction reprend d'autres fonctions vues plus haut
# Elle retourne par ailleurs le vecteur colonne T au bout d'un certain temps donné par n*delta_t
# Chaque étape (soit chaque boucle) correspond à un avancement de delta_t dans le temps
# Plus on fait de boucles, plus on se rappoche de la valeur théorique des températures à l'équilibre thermodynamique dans la colonne
# En effet, on atteint à un certain moment des températures constantes dans la colonne de terre qar on a atteint l'équilibre
# Pour rappel on considère que l'on s'intéresse à une colonne de terre dont la température est initialement nulle et dont on applique une certaine température au niveau du sol et au niveau de la dernière couche de sol)
# Ainsi, pour plus de logique, on suppose que la température imposée sur la dernière couche du sol est égale à la température initiale de la colonne
# (cela semblerait étrange d'imposer une température dans le sol à la colonne de terre)


def methode_explicite(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n):
    
    x=intervalle_discret(N)
    alpha=calcul_alpha(lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol)
    lamb=calcul_lambda(alpha,delta_x,delta_t)
    T=initialisation_T(N,T0)
    ext=extremite(N,lamb,Tg,Td)
    
    A=matrice_diag_e(N,lamb)
    
    
    for i in range(1,n+1):
        T=A.dot(T)+ext
        
    return T


# METHODE IMPLICITE

# La fonction suivante permet de créer la matrice diagonale de taille (N-1)*(N-1)
# Cette matrice sera utilisée pour calculer le vecteur T à l'étape k+1 avec la méthode implicite
# ATENTION : cette matrice diagonale est différente pour la méthode explicite

def matrice_diag_i(N,lamb):
    k = np.array([-lamb*np.ones(N-2),1+2*lamb*np.ones(N-1),-lamb*np.ones(N-2)])
    offset = [-1,0,1]
    A = diags(k,offset).toarray()
    return A

# La fonction suivante est l'implémentation de la méthode explicite
# Cette fonction reprend d'autres fonctions vues plus haut
# Elle retourne par ailleurs le vecteur colonne T au bout d'un certain temps donné par n*delta_t
# [Explications cf méthode explicite]

def methode_implicite(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n):
    
    x=intervalle_discret(N)
    alpha=calcul_alpha(lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol)
    lamb=calcul_lambda(alpha,delta_x,delta_t)
    T=initialisation_T(N,T0)
    ext=extremite(N,lamb,Tg,Td)
    
    A=matrice_diag_i(N,lamb)
    
    for i in range(1,n+1):
        T= np.linalg.solve(A,T+ext)
    return T




In [41]:
Tg=10 
Td=0 
T0=0 

N=10

delta_x=0.1   
delta_t=10    

lambda_conductivite_thermique_sol=2.1
capacite_thermique_volumique_sol=1674*10**3

n=1000


In [42]:
# Méthode explicite

T_e=methode_explicite(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n)

# Affichage des températures de la colonne de terre après un temps de n*delta_t

print('T explicite =',T_e)


T explicite = [[5.24235033e+00]
 [2.12816148e+00]
 [6.86938897e-01]
 [1.81613622e-01]
 [4.03663426e-02]
 [7.70663722e-03]
 [1.28608985e-03]
 [1.90262788e-04]
 [2.49265682e-05]]


In [43]:
# Méthode implicite

T_i=methode_implicite(N,Tg,Td,T0,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_x,delta_t,n)

# Affichage des températures de la colonne de terre après un temps de n*delta_t

print('T implicite =',T_i)

T implicite = [[5.23980440e+00]
 [2.12695699e+00]
 [6.87200628e-01]
 [1.82111234e-01]
 [4.06384602e-02]
 [7.80323060e-03]
 [1.31210387e-03]
 [1.95952499e-04]
 [2.59555921e-05]]
